In [ ]:
import numpy as np
import cv2
from skimage import morphology

def segmenter( rgb_img: np.ndarray, black_object=True) -> np.ndarray:
    move_thr = 0.3
    gray_img=cv2.cvtColor(rgb_img.copy(), cv2.COLOR_RGB2GRAY)
    thr,mask =cv2.threshold(gray_img,0,1,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    mascara = np.zeros_like(gray_img, dtype= bool)
    if black_object:
        mascara[gray_img<=thr*(1 + move_thr)] = True 
    else:
        mascara[gray_img>=thr*(1 - move_thr)] = True
            
    mascara.dtype=bool# remove_small only works with  bool datatype
    mascara=morphology.remove_small_objects(mascara,50,connectivity=1)
    mascara.dtype=np.uint8
    return mascara

# circle detector


In [ ]:
import cv2
import numpy as np
def fill_image(binary):
    src = binary.copy()   # Primero crea una copia
    mask = np.zeros([src.shape[0]+2, src.shape[1]+2, 1], np.uint8)   # Cree una máscara de acuerdo con la forma de la copia. Tenga en cuenta que la longitud y el ancho deben ser +2, y el tipo solo puede ser uint8
    cv2.floodFill(src, mask, (0, 0), (255, 255, 255), (50,50,50), (50,50,50), cv2.FLOODFILL_FIXED_RANGE)
    src = cv2.bitwise_not(src)
    filled_img =  src
    return filled_img

class Segmenter:
    def __init__(self)->None:
        self.shape = 0
    def get_detector(self):
        total_pixels = self.shape[0]*self.shape[1]
        blobParams = cv2.SimpleBlobDetector_Params()

        # # Filter by Area.
        blobParams.filterByArea = True
        blobParams.minArea = 0.005*total_pixels  # minArea may be adjusted to suit for your experiment
        blobParams.maxArea = 0.04*total_pixels # maxArea may be adjusted to suit for your experiment
        self.detector = cv2.SimpleBlobDetector_create(blobParams)
        
    def get_mask (self, rgb_img):
        # update segmenter if the shape change
        if self.shape != rgb_img.shape[:2]:
            self.shape = rgb_img.shape[:2]
            self.get_detector()
            
        points = self.detector.detect(rgb_img)
        mask = np.zeros(self.shape, dtype=np.uint8)
        print('shape',mask.shape)
        for p in points:
            x,y=p.pt
            mask=cv2.circle(mask, (int(x),int(y)), radius=int(p.size/(2)), color=(1,0,0), thickness=-1)
        return mask

# Data calibrations

In [ ]:
    import numpy as np
    class Calibration:
        depth_matrix:np.ndarray
        def __init__(self)->None:
            self.setup_values()
            
        def setup_values(self):
            self.fx = self.depth_matrix[0, 0]
            self.fy = self.depth_matrix[1, 1]
            self.depth_inv = np.linalg.inv(self.depth_matrix)
    class Calibration640_480(Calibration) :
        depth_matrix  = np.array([[389.127, 0, 321.047],
                                [0, 389.127, 241.991],
                                [0,       0,       1],
                                ])
        def __init__(self)->None:
            super().__init__()
            
        
    calibration640_480 = Calibration640_480()

    class Calibration848_480(Calibration) :
        depth_matrix  = np.array([[429.661, 0, 425.156],
                                [0, 429.661, 242.199],
                                [0,       0,       1],
                                ])
        def __init__(self)->None:
            super().__init__()

    calibration848_480 = Calibration848_480()
    calibration848_480.depth_inv  

# new calibration

In [ ]:
import numpy as np
class Calibration640_480N(Calibration) :
    depth_matrix  = np.array([[388.732, 0, 321.437],
                              [0, 388.732, 241.958],
                              [0,       0,       1],
                              ])
    def __init__(self)->None:
        super().__init__()
        
    
Calibration640_480N = Calibration640_480N()


# get reference plane. Background

In [ ]:
from sklearn import linear_model
def segmenter( rgb_img: np.ndarray, move_thr = 0.1, black_object=True) -> np.ndarray:
    gray_img =cv2.cvtColor(rgb_img.copy(), cv2.COLOR_RGB2GRAY)
    thr,mask =cv2.threshold(gray_img,0,1,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    mask = np.zeros_like(gray_img, dtype= bool)
    if black_object:
        mask[gray_img<=thr*(1 + move_thr)] = True 
    else:
        mask[gray_img>=thr*(1 - move_thr)] = True
            
    mask.dtype=bool# remove_small only works with  bool datatype
    mask=morphology.remove_small_objects(mask,50,connectivity=1)
    mask.dtype=np.uint8
    return mask

class MakeBackgroundBasic:
    shape = None
    def make_uv_cords(self, shape):
        self.shape = shape
        self.v, self.u = np.mgrid[0:shape[0],0:shape[1]]
        self.u = self.u.reshape(-1)
        self.v = self.v.reshape(-1)
        
    def get_img_points (self, depth_img,  mask):
        if self.shape != depth_img.shape:
            self.make_uv_cords(depth_img.shape)
        mask = mask.reshape(-1)
        depth_row = depth_img.reshape(-1)
        index = np.where(mask==1)
        depth_row= depth_row[index].reshape(-1,1)
        u = self.u[index]
        v = self.v[index]
        uv = np.vstack((u,v)).T
        return uv,depth_row
    
    def get_background_img(self):
        pass
    
    def save(self,path,name, background_img):
        cv2.imwrite(f'{path}{name}.png',background_img)

# From one image

In [ ]:
from typing import Optional
class MakeBackgroundOneImg(MakeBackgroundBasic):
    def get_background_img(self, depth_img:np.ndarray, mask: Optional[np.ndarray]=None):
        if mask is None:
            mask = self.segment_by_mean_depth(depth_img)
        uv, depth_row = self.get_img_points(depth_img, mask)
        model = linear_model.LinearRegression()
        # train model
        model.fit(uv,depth_row)
        UV = np.vstack((self.u,self.v)).T # shape = (u*v,2)
        background_row = model.predict(UV)
        background_img = background_row.reshape(self.shape)
        background_img = np.round(background_img,0)
        background_img = np.uint16(background_img)
        return background_img
    
    def segment_by_mean_depth(self, depth_img):
        """this will consider that the background is predominant in the scene.
        Returns:
            background_mask: _description_
        """
        depth_3 = cv2.merge((depth_img,depth_img,depth_img))# to have more points if the background is dominant, will be added more background points
        mean_threshold = depth_3.mean() - 2*depth_3.std()
        background_mask = np.zeros_like(depth_img)
        background_mask[depth_img>=mean_threshold] = 1
        return background_mask

# From multiple images

In [ ]:
import numpy as np
import cv2
from skimage import morphology
from sklearn import linear_model
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/ingelec2/Desktop/flujoBagazo/codigo/calibracion/")
from calibracion import graficar, get_names
class MakeBackground(MakeBackgroundBasic):
    def __init__(self, path)->None:
        """make background from multiple images

        Args:
            path (_type_): directory where the images are stored
        """
        self.rgb_names = get_names(path,'rgb', True)
        self.depth_names = get_names(path,'depth', True)
    
    def get_points(self):
        uv_points = []
        z_values = []
        for rgb, depth in zip(self.rgb_names, self.depth_names):
            print(rgb)
            rgb = cv2.imread(rgb)
            depth = cv2.imread(depth, cv2.IMREAD_UNCHANGED)
            if not hasattr(self, 'u'):
                self.make_uv_cords(depth.shape)
                
            mask = segmenter(rgb, black_object=True)
            img_points, depth_row = self.get_img_points(depth, mask)
              
            uv_points.append(img_points)
            z_values.append(depth_row)
        
        return np.vstack(uv_points), np.vstack(z_values)
    
    def get_background_img(self):
        uv_points, z_values= self.get_points()
        print(uv_points.shape, z_values.shape)
        model = linear_model.LinearRegression()
        # train model
        model.fit(uv_points,z_values)
        UV = np.vstack((self.u,self.v)).T # shape = (u*v,2)
        background_row = model.predict(UV)
        background_img = background_row.reshape(self.shape)
        background_img = np.round(background_img,0)
        background_img = np.uint16(background_img)
        return background_img
        

In [ ]:
# path = '/home/ingelec2/Desktop/flujoBagazo/dataset/intel/medicion/200/caja1/640_/'
path = '/home/ingelec2/Desktop/flujoBagazo/dataset/intel/medicion_after_cal/volume/200/caja1_/'
maker = MakeBackground(path)
background_img = maker.get_background_img()
background_img.shape

In [ ]:
plt.hist(background_img.reshape(-1))

In [ ]:
maker.save(path,'background',background_img)

# length measure


In [ ]:
# length measures
import numpy as np
import cv2
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/ingelec2/Desktop/flujoBagazo/codigo/calibracion/")
from calibracion import graficar, get_names, puntos
class longitud(puntos):
    # path_configuracion: dirección absoluta del lugar donde se encuentra el archivo de configuración
    #nombre_configuración: nombre del archivo configuración sin la extensión .ini
    #d:diametro de circulos en milimetros
    # nf: numero de filas del patron
    #nc: numero de circulos por fila del patron
    def __init__(self,d,nf,nc, depth_inverse):
       
        puntos.__init__(self,nf,nc)
        # variables de isntancia
        self.distancia_patron=self.formar_distancia(d)# distancia original entre puntos contiguos del patron. tiene en cuenta que los puntos estan agrupados por fila, los puntos de cada fila estan ordenados de izquierda a derecha, y las filas de arriba hacia abajo.
        self.error_promedio=0
        self.depth_inversa = depth_inverse
    # abre el par de imagenes rgb y profundidad que esten captando la misma escena
    #los nombre desben estar en el formato indicador_nombre. indicardor="rgb" para la imagen del sensor rgb
    #indicador="depth" para la imagen del sensor de profundidad. 
    #nobre debera ser el mismo para el par de imagenes que representen la misma escena.
    #pasa la imagen rgb al dominio de profundidad
    #
    def obtener_uvz(self, rgb_img, depth_img):
        ret,_,puntos_uv=self.extraer(rgb_img,0)# es una imagen en el dominio de profundidad pero su caracteristicas siguen siendo de color. no necesita mejora
        # informar si hubo detección de puntos
        print("hubo de detección de puntos?: {}".format(ret))
        #obtener cordenada z # numpy indexa fila x columna, opencv indexa ancho por lato. por lo tanto hay que invertir los indices.
        z=[]
        if ret:  
            for punto in puntos_uv:
                fila=int(np.around(punto[0][1]))
                col=int(np.around(punto[0][0]))
                valor=depth_img[fila,col]

                z.append(valor)
        z=np.array([z])
        #dibujar puntos medios sobre la imagen de profundidad
        img_puntos=self.dibujar(depth_img,puntos_uv,1)
        return ret,puntos_uv,z,img_puntos
    # la función resive puntos uv y los trasforma a puntos X,Y,Z
    # reproyecta los puntos uv encontrados en la imagen y los lleva al espacio.las cordenadas son cartesians y su origen  es el foco de la camara
    def obtener_xyz(self,puntos_uv,z):
        
        # los puntos estan almacenados en vectores fila [u,v]. para la multiplicacion matricial se los lleva a vectores tipo columna [u;v;1]
        puntos_uv=puntos_uv.reshape(-1,2) # desepaquetar. de shape(n,1,2) hacia shape(n,2)
        puntos_u=puntos_uv[:,0]# extraer todas las componentes u de los puntos
        puntos_v=puntos_uv[:,1]# #extraer todas las componentes v
        unos=np.ones(puntos_uv.shape[0])
        puntos_uv1=np.array((puntos_u,puntos_v,unos))# formar vectores columna (u;v;1)
        puntos_xy1_n=np.dot(self.depth_inversa,puntos_uv1)# calcular cordenadas normalizadas
        puntos_xyz=z*puntos_xy1_n# calular cordenadas espaciales de la forma [X;Y;Z]. son vectores columna
        #devolver puntos a forma vector fila [(x,y,z)].separando los putos
        puntos_xyz=np.concatenate((puntos_xyz[0].reshape(-1,1),puntos_xyz[1].reshape(-1,1),puntos_xyz[2].reshape(-1,1)),axis=1)
        return (puntos_xyz) #entregar array de la forma shape(n,3)
    #encuentra la distancia euclidiana entre los puntos contiguos XYZ y los compara  con la distancia de construcción de los puntos del patron
    def calcular_distancia(self,puntos_xyz):
        distancias=[]
        error_distancias=[]
        for i in range(puntos_xyz.shape[0]-1):
            vector=puntos_xyz[i+1]-puntos_xyz[i]
            distancia=np.sqrt(np.sum(np.power(vector,2)))
            distancias.append(distancia)
        distancias=np.array(distancias).reshape(-1,1)/10
        error_distancias=np.absolute(self.distancia_patron-distancias)
        error_distancias=100*error_distancias/self.distancia_patron
        self.error_promedio=np.sum(error_distancias)/error_distancias.size
        return distancias,error_distancias,self.error_promedio
    #la distancia se calcula entre dos puntos contiguos
    #d=diametro de los circulos
    #nf=numero de filas del patron
    #nc=numero de circulos por fila
    #los puntos estan agrupados por filas, los puntos de cada fila se encuentran ordenados de izquierda a derecha y las filas de arriba hacia abajo
    def formar_distancia(self,d):
        nc=self.nc-1# al ser puntos medio entre circulos se pierde un punto por cada fila
        distancias_patron=np.ones((self.nf*nc-1,1))#al ser distancia sobre puntos contiguos la informacion disminuye en un dato
        #formar distancias teniendo en cuenta la estructura del patron
        for i in range(distancias_patron.size):
            if (i+1)%nc:# puntos contiguos de una misma fila
                distancias_patron[i]=2*d
            else:#puntos contiguos, punto extremo derecho fila superior, punto extremo izquierdo fila inferior
                if (i+1)%8:# por se patron asimetrico,las distancias entre extremos cambian de una fila a otra
                    distancias_patron[i]=np.sqrt((2*(nc-1)*d-d)**2+d**2)#fila impar
                else:
                    distancias_patron[i]=np.sqrt((2*(nc-1)*d+d)**2+d**2)#fila par
        return distancias_patron

def open_pair(rgb_pathname,depth_pathname):
    rgb = cv2.imread(rgb_pathname)
    depth = cv2.imread(depth_pathname, cv2.IMREAD_UNCHANGED)
    return rgb, depth

In [ ]:

path_img = '/home/ingelec2/Desktop/flujoBagazo/dataset/intel/aligment/120/to_rgb_848_480/'
depth_inverse = calibration848_480.depth_inv
medir=longitud(100,8,5, depth_inverse)
rgb_names = get_names(path_img,'rgb', True)
depth_names = get_names(path_img,'depth', True)
list_mean_error = []
for rgb, depth in zip(rgb_names, depth_names):
    print(rgb)
    print(depth)
    rgb, depth = open_pair(rgb,depth)
    ret,puntos_uv,z,img_puntos=medir.obtener_uvz(rgb,depth)
    if ret:
            print("distancia media",z.mean())
            xyz=medir.obtener_xyz(puntos_uv,z)
            distancias,error_distancias,error_promedio=medir.calcular_distancia(xyz)
            list_mean_error.append(error_promedio)
            # for i in range(distancias.size):
            #     print("puntos {} y {} : las siguientes son unidades en milimetros".format(i,i+1))
            #     print("distancia calculada:{}".format(distancias[i]))
            #     print("distancia original:{}".format(medir.distancia_patron[i]))
            #     print("error en porcentaje :{}%".format(error_distancias[i]))
            # print("error promedio en porcentaje :{}%".format(error_promedio))
            # graficar([img_puntos],1,1,["imagen depth con puntos detectados"])
if  list_mean_error:           
    list_mean_error = np.array(list_mean_error)
    print('')
    print(f'total_mean', list_mean_error.mean())
else:
    print('check directory')

# area measure


In [ ]:
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/ingelec2/Desktop/flujoBagazo/codigo/calibracion/")
from calibracion import graficar, get_names, AreaCal, ConfigHandlerJson
from types import FunctionType
class IntelArea:
    def __init__(self, calibration_data: Calibration)->None:
        self.calibration_data = calibration_data
        self.TO_METERS = 1/10000
        self.AREA_CONSTANT = self.calibration_data.fx*self.calibration_data.fy
        area_cali = AreaCal(ConfigHandlerJson().open(path='', filename='intel'))
        self.get_area2 = area_cali.get_area
        self.ZMIN = 4000
        self.ZMAX = 22000
    def normalization(self, z_row:np.ndarray) -> np.ndarray:
        return (z_row-self.ZMIN)/(self.ZMAX-self.ZMIN)
    
    def set_model_area(self, coefs: list)->FunctionType:
        """ the coefficients must be sorted as coef[0]x²+coef[1]x+coef[2].
        Args:
        coefs (list): polynomial coefficients, if area optimization was not implemented the constant area must be inside of a list [constant_area]
        """
        def get_area(z_array: np.ndarray) -> np.ndarray:
            """
            if area_optimization was implemented, z_array must be normalized.
            check area optimization was made in the meter units [m].
            check are optimization and z_distortion has the same z_min and z_max values used into the normalization.
            """
            z_norm = self.normalization(z_array.copy())
            return np.polyval(coefs, z_norm)

        return  get_area
    
    def get_area(self, z_row:np.ndarray):
        z_row_copy = self.unify_units(z_row.copy())
        return z_row_copy/self.AREA_CONSTANT
    
    def unify_units(self, depth_img: np.ndarray) -> np.ndarray:
        """ convertion to meter units"""
        return depth_img*self.TO_METERS
    
    def get_measurement(self, z_img: np.array, mask: np.ndarray) -> float:
        """
        z_img, mask, and ref_image must have the same shape.
        example: shape=[480,640].

        Args
            z_img= this image must have de same units that background_image. 
            mask (np.ndarray): image with the same shape of z_img, where pixels=1 are the objet pixel to make the measurement.
        Returns: float: area measurement [m²]. 
        """
        #  unify dimensions to have rows elements that have 1:1 pixel position correspondence betwin all them
        #  to make this all images must have de same shape 
        z_row = z_img.reshape(-1)
        # z_row = self.unify_units(z_row)# transform to meters
        # print(f'z mean', z_row.mean())
        mask_row = mask.reshape(-1)
        # find pixel location of the object to make its measure
        index = np.where(mask_row == 1)
        z_target = z_row[index]
        # get are pixel by pixel
        area_target = self.get_area2(z_target)
        total_area = area_target.sum()
        return total_area, z_target.mean()

In [ ]:
import matplotlib.pyplot as plt 
class TestMeasure:
    def __init__(self, path_images: str,calibration_data)->None:
        self.rgb_names = get_names(path= path_images, prefix='rgb', sort= True)
        self.depth_names = get_names(path= path_images, prefix='depth', sort= True)
        self.meter = IntelArea( calibration_data)
        self.segmenter = Segmenter()

    def get_areas(self) -> np.ndarray:
        results = []
        z = []
        for rgb_name, depth_name in zip(self.rgb_names, self.depth_names):
            print(rgb_name)
            rgb = cv2.imread(rgb_name)
            depth = cv2.imread(depth_name, cv2.IMREAD_UNCHANGED)
            mask = self.segmenter.get_mask(rgb)
            print(mask.shape)
            img_mascara=cv2.bitwise_and(rgb,rgb,mask=mask)
            # print([rgb_name, depth_name])
            # graficar([img_mascara], 1,1, list_titulos=[rgb_name, depth_name])
            
            area, z_mean = self.meter.get_measurement(depth, mask)
            print('z mean', z_mean)
            print('area', area)
            z.append(z_mean)
            results.append(area)
        return np.array(results), np.array(z)
    
    def get_error(self, list_measures: np.ndarray, ground_true: float) -> np.ndarray:
        lit_error = 100*(list_measures-ground_true)/ground_true
        return lit_error
    
    def draw_hist(self, list_values: np.ndarray) -> None:
        plt.hist(list_values)
        plt.show()
    
    def analisis(self,list_error:np.ndarray, z:np.ndarray) -> None:
        self.draw_hist(list_error)
        print(f'mean error: {list_error.mean()}')
        
        plt.plot(z,list_error)
        

In [ ]:
# patron_volume = 0.46*0.55*0.362# in meters CAJA1
patron_area = np.pi*(0.1**2)
print(f'patron area: {patron_area}')
path = '/home/ingelec2/Desktop/flujoBagazo/dataset/intel/medicion_after_cal/area/area_range/'
test = TestMeasure(path,Calibration640_480N)
list_areas,z = test.get_areas()# in meters

list_errors = test.get_error(list_areas, patron_area)
test.analisis(list_errors,z) 

In [ ]:
index = np.where(z<18000)
new_z = z[index]
new_e = list_errors[index]
plt.plot(new_z, new_e)
plt.title("Error vs Profundidad")
plt.xlabel("[100 um]")
plt.ylabel("[%]")
plt.show()
plt.hist(new_e)
plt.title("Error")
plt.xlabel("[%]")
print(new_e.mean())
print

In [ ]:
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/ingelec2/Desktop/flujoBagazo/codigo/calibracion/")
from calibracion import graficar, get_names

class IntelVolume():
    
    def __init__(self, background_img: np.ndarray= None, calibration_data:Calibration= None)->None:
        self.calibration_data = calibration_data
        self.TO_METERS = 1/10000
        self.set_background(background_img)
        self.get_area = IntelArea(calibration_data).get_area2
        
        
    def set_background(self, background_img: np.ndarray) -> None:
        self.back_ground = self.unify_units(background_img)
    
    def unify_units(self, depth_img: np.ndarray) -> np.ndarray:
        """ convertion to meter units"""
        return depth_img*self.TO_METERS
    
    def get_measurement(self, z_img: np.ndarray, mask: np.ndarray) -> float:
        """
        z_img, mask, and ref_image must have the same shape.
        example: shape=[480,640].

        Args
            z_img= this image must have de same units that background_image. 
            mask (np.ndarray): image with the same shape of z_img, where pixels=1 are the objet pixel to make the measurement.
        Returns: float: volume measurement [m³]. at the moment as reference, the first significant digit is at the second decimal. example volume=0.046 m³.where 4 is the first significant digit. if the significant digit is in another position E.g. volume = 0.46  or 0.0046 it's highly probable that an error in units has happend
        """
        #  unify dimensions to have rows elements that have 1:1 pixel position correspondence betwin all them
        #  to make this all images must have de same shape 
        z_row = z_img.reshape(-1)# transform to meters
        mask_row = mask.reshape(-1)
        ref_row = self.back_ground.copy().reshape(-1)
        # find pixel location of the object to make its measure
        index = np.where(mask_row == 1)
        z_target = z_row[index]
        ref_target = ref_row[index]
        # get are pixel by pixel
        area_target = self.get_area(z_target)
         # get  pixel by pixel heigh  of target
        z_target = self.unify_units(z_target)
        height_target = ref_target-z_target
        print('heigh', height_target.mean())
        print('area', area_target.sum())
#       # get  pixel by pixel volume of target
        vol_target = height_target*area_target
        # total_vol units must be in m³
        total_vol = np.sum(vol_target)
        if total_vol < 0:
            total_vol = 0
        return total_vol

In [ ]:
import cv2
from skimage import morphology
from typing import List
import matplotlib.pyplot as plt
        
    
class TestVolume:
    def __init__(self, path_images: str, path_name_background,calibration_data, black_object = True)->None:
        self.rgb_names = get_names(path= path_images, prefix='rgb', sort= True)
        self.depth_names = get_names(path= path_images, prefix='depth', sort= True)
        background_img = cv2.imread(path_name_background, cv2.IMREAD_UNCHANGED)
        print('background distance', background_img.mean())
        self.meter = IntelVolume(background_img, calibration_data)
        self.black_object = black_object
    

    def get_volumes(self, pnt_init=None, pnt_end=[-1,-1]) -> np.ndarray:
        results = []
        for rgb_name, depth_name in zip(self.rgb_names, self.depth_names):
            print(rgb_name)
            rgb = cv2.imread(rgb_name)
            depth = cv2.imread(depth_name, cv2.IMREAD_UNCHANGED)
            
            if pnt_init is not None:# get ROI
                rgb = rgb[pnt_init[0]:pnt_end[0], pnt_init[1]:pnt_end[1]]
                depth = depth[pnt_init[0]:pnt_end[0], pnt_init[1]:pnt_end[1]]
                
            mask = segmenter(rgb, black_object= self.black_object)
            
            img_mascara=cv2.bitwise_and(rgb,rgb,mask=mask)
            # print([rgb_name, depth_name])
            # graficar([img_mascara], 1,1, list_titulos=[rgb_name, depth_name])
            
            volume = self.meter.get_measurement(depth, mask)
            print('volume', volume)
            results.append(volume)
        return np.array(results)
    
    def get_error(self, list_volumes: np.ndarray, patron_volume: float) -> np.ndarray:
        lit_error = 100*np.abs(list_volumes-patron_volume)/patron_volume
        return lit_error
    
    def draw_hist(self, list_values: np.ndarray) -> None:
        plt.hist(list_values)
        plt.title('Error absoluto')
        plt.xlabel("[%]")
        plt.show()
    
    def analisis(self,list_error:np.ndarray) -> None:
        self.draw_hist(list_error)
        print(f'mean error', list_error.mean())
        

In [ ]:
0.46*0.55

In [ ]:
patron_volume = 0.46*0.55*0.362# in meters box 1
print('ground True', patron_volume)
# patron_volume = 0.325*0.48*0.375# Box 2
# path = '/home/ingelec2/Desktop/flujoBagazo/dataset/intel/medicion/136/caja2/640/'
# path = "/home/ingelec2/Desktop/flujoBagazo/dataset/intel/medicion_after_cal/volume/caja1/137/"
path = "/home/ingelec2/Desktop/flujoBagazo/dataset/intel/medicion_after_cal/volume/200/caja1_/"
background_name = 'background.png'
path_name_background = f'{path}{background_name}'
test = TestVolume(path, path_name_background,Calibration640_480N, black_object = False)
# list_volumes = test.get_volumes()# in meters
list_volumes = test.get_volumes(pnt_init=[10,10],pnt_end=[-50,-100])# in meters

list_errors = test.get_error(list_volumes, patron_volume)
test.analisis(list_errors) 

# Volume Range

In [ ]:
import cv2
from skimage import morphology
from typing import List, Optional
import matplotlib.pyplot as plt
from calibracion import get_index

def recover(img):
        mask=img.copy()
        mask[mask<3000]=1
        mask[mask!=1]=0
        mask=mask.astype(np.uint8)
        dst = cv2.inpaint(img.copy(),mask,1,cv2.INPAINT_TELEA)
        return dst.astype(float)
    
background_img_maker = MakeBackgroundOneImg()

def get_mask_by_depth(depth_img, background_img: Optional[np.ndarray]= None,diff_thr=500):
    """consider 2 object One in front the other. the segmentation have in main that the back object is predominant and parallel to the camera.

    Args:
        diff_thr (float): mean difference in [um] between the back object and front. this could consider as the half height of the front object too
    Returns:
        mask: numpy array one chanel
    """
    if background_img is None:
        background_img =  background_img_maker.get_background_img(depth_img)
    delta_img = background_img - depth_img
    mask = np.zeros_like(depth_img, dtype= np.uint8)
   
    mask[delta_img>=diff_thr] = 1
    return mask

class VolumeInRange:
    def __init__(self, path_images: str,calibration_data, black_object = True)->None:
        self.rgb_names = get_names(path= path_images, prefix='rgb', sort= True)
        self.depth_names = get_names(path= path_images, prefix='depth', sort= True)
        
        self.meter = IntelVolume(background_img=0,calibration_data=calibration_data)
        self.black_object = black_object
        self.make_background_img = MakeBackgroundOneImg()
        self.background_img = None

    def get_volumes(self, pnt_init=None, pnt_end=[-1,-1]) -> np.ndarray:
        results = []
        z = []
        for rgb_name, depth_name in zip(self.rgb_names, self.depth_names):
            print(rgb_name)
            rgb = cv2.imread(rgb_name)
            depth = cv2.imread(depth_name, cv2.IMREAD_UNCHANGED)
            
            if pnt_init is not None:# get ROI
                rgb = rgb[pnt_init[0]:pnt_end[0], pnt_init[1]:pnt_end[1]]
                depth = depth[pnt_init[0]:pnt_end[0], pnt_init[1]:pnt_end[1]]
            depth_copy = depth.astype(float)
            depth = recover(depth)
            
            index = get_index(rgb_name, 'rgb')
            print('index:', index)
            if (index)%5==0:
                background_img = self.make_background_img.get_background_img(depth)
                self.background_img = background_img.copy()
                # graficar([background_img,], 1,1)
                self.meter.set_background(background_img)
                

            mask = get_mask_by_depth(depth_copy, background_img=self.background_img, diff_thr=1000)
            
            img_mascara=cv2.bitwise_and(rgb,rgb,mask=mask)
            # print([rgb_name, depth_name])
            graficar([img_mascara], 1,1, list_titulos=[rgb_name])
            
            volume = self.meter.get_measurement(depth, mask)
            print('volume', volume)
            results.append(volume)
            z.append(self.meter.back_ground.mean())
        return np.array(results), np.array(z)
    
    def get_error(self, list_volumes: np.ndarray, patron_volume: float) -> np.ndarray:
        lit_error = 100*np.abs(list_volumes-patron_volume)/patron_volume
        return lit_error
    
    def draw_hist(self, list_values: np.ndarray) -> None:
        plt.hist(list_values)
        plt.title('Error absoluto')
        plt.xlabel("[%]")
        plt.show()
    
    def analisis(self,list_error:np.ndarray, z) -> None:
        self.draw_hist(list_error)
        print(f'mean error', list_error.mean())
        plt.plot(z,list_error)
        plt.show()
        

In [ ]:
patron_volume = 0.2*0.22*0.19# in meters, area*depth 
print('ground True', patron_volume)
path = "/home/ingelec2/Desktop/flujoBagazo/dataset/intel/medicion_after_cal/volume/range/"
test = VolumeInRange(path,Calibration640_480N, black_object = True)
# list_volumes = test.get_volumes()# in meters
list_volumes, list_z = test.get_volumes(pnt_init=[100,100],pnt_end=[-50,-100])# in meters

list_errors = test.get_error(list_volumes, patron_volume)
test.analisis(list_errors, list_z) 